## 最終課題_オリジナルAIの構築

ディープラーニングの手法（教師あり学習の二値分類モデル）およびTensorflow・Kerasを用いて、問題を解決するAIを構築する。

> データ元 : 日本声優統計学会 https://voice-statistics.github.io/

In [3]:
import sys
print(sys.path)
print(sys.executable)
print("Python executable:", sys.executable)
print("Python path:", sys.path[0])

# ライブラリの確認
try:
    import librosa
    print(f"✓ librosa version: {librosa.__version__}")
except ImportError as e:
    print(f"✗ librosa import error: {e}")

try:
    import tensorflow as tf
    print(f"✓ tensorflow version: {tf.__version__}")
except ImportError as e:
    print(f"✗ tensorflow import error: {e}")


['C:\\dev\\python312.zip', 'C:\\dev\\DLLs', 'C:\\dev\\Lib', 'C:\\dev', 'c:\\dev\\aidev\\.venv', '', 'c:\\dev\\aidev\\.venv\\Lib\\site-packages', 'c:\\dev\\aidev\\.venv\\Lib\\site-packages\\win32', 'c:\\dev\\aidev\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\dev\\aidev\\.venv\\Lib\\site-packages\\Pythonwin']
c:\dev\aidev\.venv\Scripts\python.exe
Python executable: c:\dev\aidev\.venv\Scripts\python.exe
Python path: C:\dev\python312.zip
✗ librosa import error: No module named 'librosa'
✓ tensorflow version: 2.19.0


In [2]:
import numpy as np
import tensorflow as tf
import librosa
import sklearn
import matplotlib

print(f"numpy version: {np.__version__}")
print(f"tensorflow version: {tf.__version__}")
print(f"librosa version: {librosa.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")



ModuleNotFoundError: No module named 'librosa'

In [2]:
import sys
print("Python path:", sys.executable)
print("Python version:", sys.version)

try:
    import librosa
    print(f"librosa version: {librosa.__version__}")
    print("librosa インストール成功")
except ImportError:
    print("librosa がインストールされていません")

Python path: c:\dev\aidev\.venv\Scripts\python.exe
Python version: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
librosa がインストールされていません


In [3]:
# 事前準備_インポート

import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'librosa'

In [ ]:
# フォルダ内のデータは手動で train:test=7:3 に分けてある。

# trainフォルダのファイル一覧を取得
train_normal_files = glob.glob("./dataset_sample/train/normal/*")
train_happy_files = glob.glob("./dataset_sample/train/happy/*")

# testフォルダのファイル一覧を取得
test_normal_files = glob.glob("./dataset_sample/test/normal/*")
test_happy_files = glob.glob("./dataset_sample/test/happy/*")

In [ ]:
# それぞれのファイル数を確認
print(f'train_normal_files:', len(train_normal_files))
print(f'train_happy_files:', len(train_happy_files))
print(f'test_normal_files:', len(test_normal_files))
print(f'test_happy_files:', len(test_happy_files))

**※以下、生成分**

In [ ]:
# 音声特徴量抽出関数
def extract_features(file_path, max_length=100):
    """
    音声ファイルから特徴量を抽出
    """
    try:
        # 音声ファイル読み込み
        audio, sample_rate = librosa.load(file_path, sr=22050)
        
        # MFCC特徴量抽出
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        
        # 固定長に調整
        if mfccs.shape[1] > max_length:
            mfccs = mfccs[:, :max_length]
        else:
            mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
        
        return mfccs.flatten()
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [ ]:
# データセット準備
def prepare_dataset(data_dir):
    """
    データセットを準備
    data_dir/
    ├── normal/     # 普通の音声ファイル
    └── joy/        # 喜びの音声ファイル
    """
    features = []
    labels = []
    
    # 普通の音声データ（ラベル: 0）
    normal_dir = os.path.join(data_dir, 'normal')
    if os.path.exists(normal_dir):
        for file_name in os.listdir(normal_dir):
            if file_name.endswith(('.wav', '.mp3', '.flac')):
                file_path = os.path.join(normal_dir, file_name)
                feature = extract_features(file_path)
                if feature is not None:
                    features.append(feature)
                    labels.append(0)  # 普通
    
    # 喜びの音声データ（ラベル: 1）
    joy_dir = os.path.join(data_dir, 'joy')
    if os.path.exists(joy_dir):
        for file_name in os.listdir(joy_dir):
            if file_name.endswith(('.wav', '.mp3', '.flac')):
                file_path = os.path.join(joy_dir, file_name)
                feature = extract_features(file_path)
                if feature is not None:
                    features.append(feature)
                    labels.append(1)  # 喜び
    
    return np.array(features), np.array(labels)

In [ ]:
# モデル構築
def create_model(input_shape):
    """
    二値分類用のニューラルネットワークモデル
    """
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(input_shape,)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid')  # 二値分類用
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# メイン処理
def main():
    # データディレクトリのパス（適宜変更してください）
    data_directory = "audio_data"
    
    print("データセットを準備中...")
    X, y = prepare_dataset(data_directory)
    
    if len(X) == 0:
        print("音声データが見つかりません。データディレクトリを確認してください。")
        return
    
    print(f"データ数: {len(X)}")
    print(f"普通: {np.sum(y == 0)}件, 喜び: {np.sum(y == 1)}件")
    
    # データの正規化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 訓練・テストデータ分割
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"訓練データ: {len(X_train)}, テストデータ: {len(X_test)}")
    
    # モデル作成
    model = create_model(X_train.shape[1])
    print(model.summary())
    
    # コールバック設定
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True
    )
    
    # モデル訓練
    print("モデルを訓練中...")
    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # モデル評価
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"テスト精度: {test_accuracy:.4f}")
    
    # 予測例
    predictions = model.predict(X_test)
    predicted_classes = (predictions > 0.5).astype(int)
    
    print("\n予測結果の例:")
    for i in range(min(5, len(X_test))):
        actual = "喜び" if y_test[i] == 1 else "普通"
        predicted = "喜び" if predicted_classes[i] == 1 else "普通"
        confidence = predictions[i][0]
        print(f"実際: {actual}, 予測: {predicted}, 信頼度: {confidence:.4f}")
    
    # 学習履歴の可視化
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # モデル保存
    model.save('emotion_classifier.h5')
    print("モデルを 'emotion_classifier.h5' として保存しました")